# **Examen Coding Dojo**

Versión C – Analizando los Salarios en Ciencia de Datos en 2023

# **Alumno:** Cristian Lugo

Requisitos:

Tu tarea es limpiar y explorar un dataset que contiene información sobre los salarios en el campo de la ciencia de datos para el año 2023. Este análisis es crucial para entender las tendencias salariales y los factores que influyen en las diferencias de salarios en esta industria.


In [ ]:
# Importar las bibliotecas necesarias

import numpy as np    # Numpy se utiliza para operaciones matemáticas
import pandas as pd   # Pandas se utiliza para la manipulación y análisis de datos
import matplotlib.pyplot as plt  # Matplotlib se utiliza para visualizaciones básicas
import seaborn as sns  # Seaborn  se utiliza para visualizaciones más avanzadas

#El dataset ha sido descargado desde la Página de kaggle. Puedes encontrar el dataset en el siguiente link: https://www.kaggle.com/datasets/arnabchaki/data-science-salaries-2023
# EL archivo ha sido descargado con el nombre: ds_salaries.csv


# Paso 1: Cargar el dataset desde un archivo local
# En este paso, cargamos el archivo CSV desde la carpeta donde se encuentra el dataset.
# El método pd.read_csv() se usa para leer archivos CSV.

from google.colab import files
# Esto abrirá una ventana para que selecciones el archivo en tu computadora
uploaded = files.upload()   # Habilita la opción para seleccionar el archivo a ser cargado

file_name = next(iter(uploaded))  # Obtiene el nombre del primer archivo subido
# Carga el archivo CSV en un DataFrame de pandas.
df = pd.read_csv(file_name)

# Ver las primeras 10 filas para asegurarse de que los datos se han cargado correctamente
# Esto es simplemente una corroboración de que los datos se han cargado correctamente
print("Primeras filas del dataset:")
print(df.head(10))


# Paso 2: Eliminar duplicados
# Los duplicados en los datos pueden generar sesgos en el análisis.
# Inicialmente se encuentran 3755 filas
# El método 'drop_duplicates()' elimina cualquier fila duplicada en el DataFrame.

df.drop_duplicates(inplace=True)  # Elimina filas duplicadas si las hay

# Verificación para comprobar si se han eliminado duplicados correctamente
print(f"Total de filas después de eliminar duplicados: {len(df)}")
# Se han eliminado 1171 filas duplicadas

# Paso 3: Verificación y ajuste de tipos de datos
# Asegúrate de que todas las columnas tengan los tipos de datos correctos.
# Por ejemplo, los salarios deben ser numéricos, mientras que otras columnas pueden ser cadenas.

print("\nTipos de datos de cada columna antes de los ajustes:")
print(df.dtypes)  # Muestra los tipos de datos originales

# Asegurarnos de que los valores de la columna 'work_year','salary', 'salary_in_usd', 'remote_ratio' sean numéricos
# Si algún valor no puede convertirse, se marcará como NaN (valor no disponible)
df['work_year'] = pd.to_numeric(df['work_year'], errors='coerce')
df['salary'] = pd.to_numeric(df['salary'], errors='coerce')
df['salary_in_usd'] = pd.to_numeric(df['salary_in_usd'], errors='coerce')
df['remote_ratio'] = pd.to_numeric(df['remote_ratio'], errors='coerce')


# Paso 4: Consistencia en valores categóricos
# Las columnas categóricas pueden tener inconsistencias en la forma de escribir las categorías. Por ejemplo: ‘Junior’, ‘junior’, ‘JUNIOR’
# Estandarizamos los valores de la columna 'experience_level' para que todo esté en mayúsculas.

df['experience_level'] = df['experience_level'].str.upper()  # Convierte todos los valores a mayúsculas.
df['employment_type'] = df['employment_type'].str.upper()
df['job_title'] = df['job_title'].str.upper()
df['salary_currency'] = df['salary_currency'].str.upper()
df['employee_residence'] = df['employee_residence'].str.upper()
df['company_location'] = df['company_location'].str.upper()
df['company_size'] = df['company_size'].str.upper()


# Paso 5: Manejo de valores faltantes
# Usamo la "media" para valores numéricos y "desconocido" en valores categóricos.

print("\nValores faltantes por columna:")
print(df.isnull().sum())  # Verifica cuántos valores faltantes hay en cada columna

# Rellenamos los valores numéricos faltantes, con la media
df['salary'] = df['salary'].fillna(df['salary'].mean())
df['salary_in_usd'] = df['salary_in_usd'].fillna(df['salary_in_usd'].mean())
df['remote_ratio'] = df['remote_ratio'].fillna(df['remote_ratio'].mean())

# Rellenamos valores faltantes en columnas categóricas con 'desconocido'
df.fillna('desconocido', inplace=True)


# Paso 6: Detección de datos anómalos
# En algunos casos, los datos pueden contener valores inusuales o anómalos.
# Aquí nos aseguramos de que los salarios en dolares estén dentro de un rango razonable (por ejemplo, entre 10,000 USD y 300,000 USD). Dependiendo del IQR

# Verificar si hay valores atípicos en la columna 'salary_in_usd'
# Calcular el rango intercuartílico (IQR)
# El IQR nos ayudará a identificar los valores atípicos.
# Q1 es el percentil 25 (primer cuartil) y Q3 es el percentil 75 (tercer cuartil)
Q1 = df['salary_in_usd'].quantile(0.25)
Q3 = df['salary_in_usd'].quantile(0.75)
IQR = Q3 - Q1  # El rango intercuartílico es la diferencia entre el tercer y el primer cuartil

# Definir los límites para detectar los valores atípicos
# Utilizamos el IQR para calcular los límites superior e inferior
# Los valores por debajo del límite inferior o por encima del límite superior serán considerados atípicos
lower_bound = Q1 - 1.5 * IQR  # Límite inferior
upper_bound = Q3 + 1.5 * IQR  # Límite superior

# Ajustar los valores atípicos
# Reemplazamos cualquier valor menor que el límite inferior por el límite inferior
# Reemplazamos cualquier valor mayor que el límite superior por el límite superior
df['salary_in_usd_adjusted'] = df['salary_in_usd'].apply(
    lambda x: upper_bound if x > upper_bound else (lower_bound if x < lower_bound else x)
)


# Paso 7: Visualización exploratoria de datos
# Ahora que los datos están limpios, comenzamos la exploración visual para identificar patrones.
# Visualización univariada 1: Histograma de salarios ajustados
plt.figure(figsize=(8,6))
sns.histplot(df['salary_in_usd_adjusted'], bins=20, kde=True)
plt.title('Distribución de Salarios Ajustados en USD')
plt.xlabel('Salario en USD')
plt.ylabel('Frecuencia')
plt.show()

# Interpretación: Esta visualización nos muestra la distribución de los salarios ajustados. La mayoría de los salarios se concentran en un rango específico. Entre '100000' y '150000' USD

# Visualización univariada 2: Creamos un gráfico de barras que muestra el número de empleados por nivel de experiencia.

plt.figure(figsize=(10, 6))  # Definir el tamaño de la figura
sns.countplot(x='experience_level', data=df)  # Gráfico de barras para niveles de experiencia
plt.title('Conteo por Nivel de Experiencia')
plt.xlabel('Nivel de Experiencia')
plt.ylabel('Frecuencia')
plt.show()

# Interpretación: El nivel 'senior' parece ser el más común en el dataset.



# Paso 8: Visualización multivariada
# Visualización multivariada 1: Gráfico de caja de salarios por nivel de experiencia.

plt.figure(figsize=(10, 6))
sns.boxplot(x='experience_level', y='salary_in_usd_adjusted', data=df)
plt.title('Distribución de Salarios por Nivel de Experiencia')
plt.xlabel('Nivel de Experiencia')
plt.ylabel('Salario')
plt.show()
# Interpretación: A medida que el nivel de experiencia aumenta, también lo hacen los salarios.


# Visualización multivariada 2: Gráfico de caja de salarios por tamaño de la empresa.

plt.figure(figsize=(10, 6))
sns.boxplot(x='company_size', y='salary', data=df)
plt.title('Distribución de Salarios por Tamaño de Empresa')
plt.xlabel('Tamaño de la Empresa')
plt.ylabel('Salario')
plt.show()
# Interpretación: Las empresas más grandes tienden a pagar más que las más pequeñas.


# Paso 9: Estadísticas descriptivas
# Por último, mostramos las estadísticas descriptivas del dataset para obtener una vista general de los datos numéricos.

print("\nEstadísticas descriptivas del dataset:")
print(df.describe().round(2))  # Muestra estadísticas como media, desviación estándar, etc.

# Identificación de tendencias:
# Basado en los análisis y visualizaciones anteriores, podemos concluir que:
# 1. Los empleados con mayor nivel de experiencia ganan salarios más altos.
# 2. Las empresas de mayor tamaño suelen ofrecer mejores salarios en comparación con empresas pequeñas.



